# LSEG Data Libraries API

**Portfolio Selection**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial shows

* how to retrieve historical data across asset classes via the Eikon Data API,
* how to work with such data using `pandas`, `Plotly` and `Cufflinks` and
* how to compose and analyze portfolios with regard to their expected return and volatility.

## Portfolio Selection

Markowitz(1952): “Portfolio Selection”:

> “Various reasons recommend the use of the expected return-variance of return rule, both as a hypothesis to explain well-established investment behavior and as a maxim to guide one's own action.”

In what follows, the composition of a portfolio made up of different risky assets is analyzed in light of the **Mean-Variance Portfolio Theory (MVP)** of Markowitz (1952), i.e. with regard to the resulting **expected portfolio return** and **expected portfolio volatility** (instead of variance).

## Importing Required Packages

In [1]:
import math
import lseg.data as ld  # Importing LSEG data libraries
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
import scipy.optimize as sco  # optimization routines
cf.set_config_file(offline=True)  # set the plotting mode to offline

The following **Python and package versions** are used.

In [2]:
import sys
print(sys.version)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [3]:
ld.__version__

'2.0.0'

In [4]:
np.__version__

'1.22.1'

In [5]:
pd.__version__

'1.3.5'

In [6]:
cf.__version__

'0.17.3'

## Opening an LSEG data libraries session

In [3]:
ld.open_session()

<lseg.data.session.Definition object at 0x26f626c7b80 {name='workspace'}>

## Retrieving Cross-Asset Data

We first define a **small universe of `RICS`** for which to retrieve data.

In [4]:
rics = [
    'AAPL.O',  # Apple stock
    'AMZN.O',  # Amazon stock
    'SPY',  # S&P 500 ETF
    'GLD',  # Gold ETF
]

Second, **end-of-day (EOD) data** is retrieved.

In [5]:
data = ld.get_history(rics,  # the RICs
                      fields='TRDPRC_1',  # the required fields
                      start='2017-01-01',  # start date
                      end='2018-02-16')  # end date

In [6]:
ld.close_session()

In [17]:
data.head()  # first five rows

TRDPRC_1     AAPL.O   AMZN.O     SPY     GLD
Date                                        
2017-01-03  29.0375  37.6835  225.24  110.47
2017-01-04   29.005   37.859  226.58  110.86
2017-01-05  29.1525  39.0225   226.4  112.58
2017-01-06  29.4775  39.7995  227.21  111.75
2017-01-09  29.7475   39.846  226.46  112.67

In [18]:
data.tail()  # final five rows

TRDPRC_1     AAPL.O   AMZN.O     SPY     GLD
Date                                        
2018-02-12  40.6775  69.3115  265.34  125.37
2018-02-13   41.085  70.7255   266.0  126.08
2018-02-14  41.8425  72.5525  269.59  128.23
2018-02-15  43.2475   73.088  273.03  128.38
2018-02-16  43.1075  72.4345  273.11  127.96

Only complete data rows are selected.

In [19]:
data.dropna(inplace=True)  # deletes rows with NaN values

In [20]:
data.info()  # DataFrame meta information

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 284 entries, 2017-01-03 to 2018-02-16
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL.O  284 non-null    Float64
 1   AMZN.O  284 non-null    Float64
 2   SPY     284 non-null    Float64
 3   GLD     284 non-null    Float64
dtypes: Float64(4)
memory usage: 12.2 KB


In [21]:
data.normalize().iplot(kind='lines')

## Statistics for Single Instruments

We calculate the **log returns** in vectorized fashion for all financial instruments and all days available.

In [7]:
rets = np.log(data / data.shift(1))  # log returns in vectorized fashion
rets.dropna(inplace=True)  # deletes rows with NaN values

In [23]:
rets.head()

TRDPRC_1      AAPL.O    AMZN.O       SPY       GLD
Date                                              
2017-01-04  -0.00112  0.004646  0.005932  0.003524
2017-01-05  0.005072   0.03027 -0.000795  0.015396
2017-01-06  0.011087  0.019716  0.003571   -0.0074
2017-01-09  0.009118  0.001168 -0.003306  0.008199
2017-01-10  0.001008 -0.001281       0.0  0.004251

In MVP, the **average returns** of the financial instruments play an important role since they are used to approximate the **expected returns**.

In [9]:
rets.mean()  # daily mean returns

TRDPRC_1
AAPL.O    0.001396
AMZN.O    0.002309
SPY       0.000681
GLD       0.000519
dtype: float64

In [10]:
rets.mean() * 252  # annualied mean returns

TRDPRC_1
AAPL.O    0.351828
AMZN.O    0.581880
SPY       0.171599
GLD       0.130874
dtype: float64

In [11]:
(rets.mean() * 252).iplot(kind='bar')

On the other hand, the **historical volatility** (standard deviation of returns) plays an important role, although it is not used directly.

In [35]:
rets.std()  # daily volatilities

TRDPRC_1
AAPL.O    0.012133
AMZN.O    0.013946
SPY       0.006010
GLD       0.006269
dtype: float64

In [36]:
rets.std() * math.sqrt(252)  # annualized volatilities

TRDPRC_1
AAPL.O    0.192603
AMZN.O    0.221386
SPY       0.095412
GLD       0.099521
dtype: float64

In [37]:
(rets.std() * math.sqrt(252)).iplot(kind='bar')

## Portfolio Statistics

Assume a portfolio composed of **all financial instruments** with **equal weighting**.

In [38]:
weights = len(rics) * [1 / len(rics)]
weights

[0.25, 0.25, 0.25, 0.25]

The **expected portfolio return** according to MVP is the dot product of the expected returns and the weights.

In [39]:
np.dot(rets.mean() * 252, weights)

0.30904530642851963

In [40]:
def portfolio_return(symbols, weights):
    return np.dot(rets[symbols].mean() * 252, weights)

In [41]:
portfolio_return(rics, weights)

0.30904530642851963

`pandas` allows to derive the **covariance matrix** with a single method call. It plays a major role in the theory of Markowitz (1952) in that it accounts for **diversification** effects.

In [42]:
data.cov() * 252  # annualized covariance matrix by column

TRDPRC_1        AAPL.O        AMZN.O           SPY          GLD
TRDPRC_1                                                       
AAPL.O     3679.806010   6239.000965  12203.632285  2619.901119
AMZN.O     6239.000965  16061.970983  26893.128699  5196.743242
SPY       12203.632285  26893.128699  51693.162055  9651.703778
GLD        2619.901119   5196.743242   9651.703778  3398.295109

With the covariance matrix, **expected portfolio variance** is calculated as follows.

In [43]:
np.dot(weights, np.dot(rets.cov() * 252, weights))

0.011367596012406708

Accordingly, **expected portfolio volatility** is then given by:

In [44]:
math.sqrt(np.dot(weights, np.dot(rets.cov() * 252, weights)))

0.10661892895919893

In [45]:
def portfolio_volatility(symbols, weights):
    return math.sqrt(np.dot(weights, np.dot(rets[symbols].cov() * 252, weights)))

In [46]:
portfolio_volatility(rics, weights)

0.10661892895919893

## Simulating Portfolio Compositions &mdash; Two Instruments

To get started, consider just two financial instruments for which **portfolio compositions** are simulated that add up to 100% (= 1).

In [47]:
fis = ['AAPL.O', 'AMZN.O']

In [48]:
w = np.random.random((500, len(fis)))  # random portfolio compositions ...

In [49]:
w[:5].round(2)  # ... that do not yet add up to 100%

array([[0.26, 0.13],
       [0.31, 0.39],
       [0.07, 0.52],
       [0.56, 0.2 ],
       [0.22, 0.82]])

In [50]:
w = (w.T / w.sum(axis=1)).T  # normalization ...

In [51]:
w[:5].round(2)  # ... let's the random numbers add up to 100%

array([[0.67, 0.33],
       [0.45, 0.55],
       [0.12, 0.88],
       [0.74, 0.26],
       [0.21, 0.79]])

In [52]:
w.sum(axis=1)[:5] # ... let's the random numbers add up to 100%

array([1., 1., 1., 1., 1.])

Given these random composition, the resulting **portfolio statistics** can be derived.

In [53]:
mvp = [(portfolio_volatility(fis, weights),
       portfolio_return(fis, weights))
         for weights in w]

In [54]:
mvp = pd.DataFrame(np.array(mvp), columns=['volatility', 'return'])
mvp.iloc[:5]

volatility    return
0    0.176388  0.427911
1    0.180627  0.479371
2    0.206949  0.554178
3    0.177652  0.411828
4    0.197572  0.533348

These results can then be **visualized**.

In [55]:
mvp.iplot(x='volatility', y='return', kind='scatter', mode='markers', color='red')

## Simulating Portfolio Compositions &mdash; All Instruments

Second, consider portfolio compositions for **all financial instruments**. The code is basically the same.

In [56]:
w = np.random.random((2500, len(rics)))  # random portfolio compositions ...

In [57]:
w[:5].round(2)  # ... that do not yet add up to 100%

array([[0.89, 0.09, 0.37, 0.49],
       [0.94, 0.16, 0.86, 0.37],
       [0.98, 0.56, 0.66, 0.97],
       [0.88, 0.5 , 0.75, 0.99],
       [0.54, 0.64, 0.56, 0.65]])

In [58]:
w = (w.T / w.sum(axis=1)).T  # normalization ...

In [59]:
w[:5].round(2)  # ... let's the random numbers add up to 100%

array([[0.48, 0.05, 0.2 , 0.26],
       [0.4 , 0.07, 0.37, 0.16],
       [0.31, 0.18, 0.21, 0.31],
       [0.28, 0.16, 0.24, 0.32],
       [0.23, 0.27, 0.23, 0.27]])

In [60]:
w.sum(axis=1)[:5] # ... let's the random numbers add up to 100%

array([1., 1., 1., 1., 1.])

Given these random composition, the resulting **portfolio statistics** can be derived.

In [61]:
mvp = [(portfolio_volatility(rics, weights),
       portfolio_return(rics, weights))
         for weights in w]

In [62]:
mvp = pd.DataFrame(np.array(mvp), columns=['volatility', 'return'])
mvp.iloc[:5]

volatility    return
0    0.113006  0.267715
1    0.111249  0.265903
2    0.101910  0.287581
3    0.097126  0.275657
4    0.105878  0.311843

These results can then be **visualized**.

In [63]:
mvp.iplot(x='volatility', y='return', kind='scatter', mode='markers', color='red')

## Minimum Volatility Portfolio

Let us now derive the portfolio composition that **minimizes the expected volatility** of the portfolio. First, the **boundary conditions** for the single weights (between 0 and 1).

In [64]:
bounds = len(rics) * [(0, 1)]

In [65]:
bounds

[(0, 1), (0, 1), (0, 1), (0, 1)]

Second, the condition that the **weights add up to 100%**.

In [66]:
constraints = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1} 

Third, the **function to be minimized**. This is the function `portfolio_volatility()` from above.

These three elements are used in combination with the **general minimizer function `sco.minimize()`** to derive the minimum volatility portfolio.

In [67]:
res = sco.minimize(lambda x: portfolio_volatility(rics, x),  # function to be minized
                   len(rics) * [1 / len(rics)],  # initial guess
                   bounds=bounds,  # boundary conditions
                   constraints=constraints  # single equality constraint
                  )

The **results** are:

In [68]:
res

     fun: 0.06476360145184672
     jac: array([0.08022096, 0.07191757, 0.06489603, 0.0646203 ])
 message: 'Optimization terminated successfully'
    nfev: 35
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.       , 0.       , 0.5197279, 0.4802721])

In [69]:
res['fun']  # minimum volatility

0.06476360145184672

In [70]:
for r in zip(rics, res['x']):
    print('%7s | %7.3f' % (r[0], r[1])) # optimal portfolio composition

 AAPL.O |   0.000
 AMZN.O |   0.000
    SPY |   0.520
    GLD |   0.480


## Conclusions

Based on this tutorial, we can conclude that

* it is easy to retrieve **historical end-of-day across asset classes** via the LSEG Data Libraries,
* `Plotly` and `Cufflinks` make **financial data visualization** convenient and
* `pandas` is a powerful data analysis tool to implement **financial algorithms** efficiently, such as the portfolio selection approach from Markowitz (1952).

## LSEG Data Libraries API Developer Resources

* [Overview](https://developers.thomsonreuters.com/eikon-data-apis) 
* [Quick Start ](https://developers.thomsonreuters.com/eikon-data-apis/quick-start)
* [Documentation](https://developers.thomsonreuters.com/eikon-data-apis/docs)
* [Downloads](https://developers.thomsonreuters.com/eikon-data-apis/downloads)
* [Tutorials](https://developers.thomsonreuters.com/eikon-data-apis/learning)
* [Q&A Forums](https://developers.thomsonreuters.com/eikon-data-apis/qa) 

Data Item Browser Application: Type `DIB` into Workspace Search Bar.